# Data preprocess

Import data and required package 

In [1]:
import pandas as pd

df_train = pd.read_csv("dataset/aug_train.csv")
df_test = pd.read_csv("dataset/aug_test.csv")

In [5]:
print(df_train.head())

   enrollee_id  city_development_index gender      relevent_experience  \
0         8949                   0.920   Male  Has relevent experience   
1        29725                   0.776   Male   No relevent experience   
2        11561                   0.624    NaN   No relevent experience   
3        33241                   0.789    NaN   No relevent experience   
4          666                   0.767   Male  Has relevent experience   

  enrolled_university education_level major_discipline experience  \
0       no_enrollment        Graduate             STEM        >20   
1       no_enrollment        Graduate             STEM         15   
2    Full time course        Graduate             STEM          5   
3                 NaN        Graduate  Business Degree         <1   
4       no_enrollment         Masters             STEM        >20   

  company_size    company_type last_new_job  training_hours  target  
0          NaN             NaN            1              36     1.0  


In [3]:
print(df_test.head())

   enrollee_id      city  city_development_index  gender  \
0        32403   city_41                   0.827    Male   
1         9858  city_103                   0.920  Female   
2        31806   city_21                   0.624    Male   
3        27385   city_13                   0.827    Male   
4        27724  city_103                   0.920    Male   

       relevent_experience enrolled_university education_level  \
0  Has relevent experience    Full time course        Graduate   
1  Has relevent experience       no_enrollment        Graduate   
2   No relevent experience       no_enrollment     High School   
3  Has relevent experience       no_enrollment         Masters   
4  Has relevent experience       no_enrollment        Graduate   

  major_discipline experience company_size company_type last_new_job  \
0             STEM          9          <10          NaN            1   
1             STEM          5          NaN      Pvt Ltd            1   
2              NaN        

In [4]:
df_train = df_train.drop(columns=["city"])
df_test = df_test.drop(columns=["city"])